In [1]:
import torch
from PIL import Image
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import cv2
import os
from sklearn import metrics
from PIL import Image
from data.imgaug import GetTransforms
from data.utils import transform
np.random.seed(0)
from easydict import EasyDict as edict
import json
cfg_path = 'E:/Chexpert/config/example.json'
import sys
import os
import argparse
import logging
import json
import time
import subprocess
from shutil import copyfile
from tensorboardX import SummaryWriter
import torch.nn as nn
from torch.optim import SGD, Adadelta, Adagrad, Adam, RMSprop
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
import tqdm

# Data Loader

In [2]:
# import numpy as np
# from torch.utils.data import Dataset
# import cv2
# import os
# from PIL import Image
# from data.imgaug import GetTransforms
# from data.utils import transform
# np.random.seed(0)
# from easydict import EasyDict as edict
# import json
# cfg_path = 'E:/Chexpert/config/example.json'
# with open(cfg_path) as f:
#     cfg = edict(json.load(f))

# class ImageDataset(Dataset):
#     def __init__(self, label_path, cfg, mode='train'):
#         self.cfg = cfg
#         self._label_header = None
#         self._image_paths = []
#         self._labels = []
#         self._mode = mode
#         self.dict = [{'1.0': '1', '': '0', '0.0': '0', '-1.0': '0'},
#                      {'1.0': '1', '': '0', '0.0': '0', '-1.0': '1'}, ]
#         with open(label_path) as f:
#             header = f.readline().strip('\n').split(',')
#             self._label_header = [
#                 header[7],
#                 header[10],
#                 header[11],
#                 header[13],
#                 header[15]]
#             for line in f:
#                 labels = []
#                 fields = line.strip('\n').split(',')
#                 image_path = fields[0]
#                 flg_enhance = False  # bujhi nai
#                 for index, value in enumerate(fields[5:]):
#                     if index == 5 or index == 8:
#                         labels.append(self.dict[1].get(value))
#                         if self.dict[1].get(
#                                 value) == '1' and \
#                                 self.cfg.enhance_index.count(index) > 0:
#                             flg_enhance = True
#                     elif index == 2 or index == 6 or index == 10:
#                         labels.append(self.dict[0].get(value))
#                         if self.dict[0].get(
#                                 value) == '1' and \
#                                 self.cfg.enhance_index.count(index) > 0:
#                             flg_enhance = True
#                 # labels = ([self.dict.get(n, n) for n in fields[5:]])
#                 labels = list(map(int, labels))
#                 self._image_paths.append(image_path)
#                 assert os.path.exists(image_path), image_path
#                 self._labels.append(labels)
#                 if flg_enhance and self._mode == 'train':
#                     for i in range(self.cfg.enhance_times):
#                         self._image_paths.append(image_path)
#                         self._labels.append(labels)
#         self._num_image = len(self._image_paths)

#     def __len__(self):
#         return self._num_image

#     def __getitem__(self, idx):
#         image = cv2.imread(self._image_paths[idx], 0)
#         image = Image.fromarray(image)
#         if self._mode == 'train':
#             image = GetTransforms(image, type=self.cfg.use_transforms_type)
#         image = np.array(image)
#         image = transform(image, self.cfg)
#         labels = np.array(self._labels[idx]).astype(np.float32)

#         path = self._image_paths[idx]
#         print(path)
#         if self._mode == 'train' or self._mode == 'dev':
#             return (image, labels)
#         elif self._mode == 'test':
#             return (image, path)
#         elif self._mode == 'heatmap':
#             return (image, path, labels)
#         else:
#             raise Exception('Unknown mode : {}'.format(self._mode))


In [3]:
# dataset = ImageDataset('E:/Chexpert/config/train.csv',cfg)
# a,b = dataset[1]

In [4]:
# a,b = dataset[39]
# b

In [5]:
# import albumentations
# import albumentations.pytorch

In [9]:
data_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
data_aug_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=(-180,180)),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2)),#new
        transforms.RandomAutocontrast(p=0.1),

        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5])
    ])
class CheXDataset(Dataset):
    def __init__(self, csv_path, diseases,transform=None, make_one=1, mode='train'):
        self.csv_path = csv_path
        self.df = pd.read_csv(self.csv_path)
        self.df = self.df.fillna(0)
        self.make_one = make_one
        header = list(df.columns)
        self._label_header = [
                header[7],
                header[10],
                header[11],
                header[13],
                header[15]]
        self.defult_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
        self.transform = transform
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = Image.open(self.df['Path'].values[idx]).convert('RGB') 
        if self.transform is not None:
            image = self.transform(image)
        else :
            image = self.defult_transform(image)
            
  
        labels = np.zeros(len(diseases))
        
        if int(self.df['No Finding'].values[idx]) == 1  :
            labels = labels
        else:
            for i in range(len(labels)):
                if self.df[diseases[i]].values[idx] != -1 :
                    labels[i] = self.df[diseases[i]].values[idx]
                elif self.make_one :
                    labels[i] = 1
                else :
                    labels[i] = 0  
                
                
        return image,labels

In [10]:
import pandas as pd
df = pd.read_csv('E:/Chexpert/config/train.csv')

In [11]:
csv_path = 'E:/Chexpert/config/train.csv'
diseases = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural_Effusion']
dataset = CheXDataset(csv_path, diseases, data_aug_transforms)

In [12]:
def get_loss(output, target, index, device, cfg):
    if cfg.criterion == 'BCE':
        for num_class in cfg.num_classes:
            assert num_class == 1
        target = target[:, index].view(-1)
        pos_weight = torch.from_numpy(
            np.array(cfg.pos_weight,
                     dtype=np.float32)).to(device).type_as(target)
        if cfg.batch_weight:
            if target.sum() == 0:
                loss = torch.tensor(0., requires_grad=True).to(device)
            else:
                weight = (target.size()[0] - target.sum()) / target.sum()
                loss = F.binary_cross_entropy_with_logits(
                    output[index].view(-1), target, pos_weight=weight)
        else:
            loss = F.binary_cross_entropy_with_logits(
                output[index].view(-1), target, pos_weight=pos_weight[index])

        label = torch.sigmoid(output[index].view(-1)).ge(0.5).float()
        acc = (target == label).float().sum() / len(label)
    else:
        raise Exception('Unknown criterion : {}'.format(cfg.criterion))

    return (loss, acc)
def lr_schedule(lr, lr_factor, epoch_now, lr_epochs):
    """
    Learning rate schedule with respect to epoch
    lr: float, initial learning rate
    lr_factor: float, decreasing factor every epoch_lr
    epoch_now: int, the current epoch
    lr_epochs: list of int, decreasing every epoch in lr_epochs
    return: lr, float, scheduled learning rate.
    """
    count = 0
    for epoch in lr_epochs:
        if epoch_now >= epoch:
            count += 1
            continue

        break

    return lr * np.power(lr_factor, count)

In [13]:
 from easydict import EasyDict as edict
cfg = {
     "train_csv": "E:/Chexpert/config/train.csv",
    "dev_csv": "E:/Chexpert/config/dev.csv",
    "backbone": "densenet121",
    "width": 256,
    "height": 256,
    "long_side": 256,
    "fix_ratio": True,
    "pixel_mean": 128.0,
    "pixel_std": 64.0,
    "use_pixel_std": True,
    "use_equalizeHist": True,
    "use_transforms_type": "Aug",
    "gaussian_blur": 3,
    "border_pad": "pixel_mean",
    "num_classes": [1,1,1,1,1],
    "batch_weight": True,
    "enhance_index": [2,6],
    "enhance_times": 1,
    "pos_weight": [1,1,1,1,1],
    "train_batch_size": 16,
    "dev_batch_size": 1,
    "pretrained": True,
    "log_every": 10,
    "test_every": 100,
    "epoch": 3,
    "norm_type": "BatchNorm",
    "global_pool": "PCAM",
    "fc_bn": False,
    "attention_map": "None",
    "lse_gamma": 0.5,
    "fc_drop": 0,
    "optimizer": "Adam",
    "criterion": "BCE",
    "lr": 0.0001,
    "lr_factor": 0.1,
    "lr_epochs": [2],
    "momentum": 0.9,
    "weight_decay": 0.0,
    "best_target": "auc",
    "save_top_k": 30,
    "save_index": [0,1,2,3,4],
    'save_path' : 'Checkpoints',
    'logtofile' : True,
    'resume' : False,
    'pre_train' : None,
    'verbose' :  False
}
cfg  = edict(cfg)

In [14]:
cfg.save_path

'Checkpoints'

In [15]:



def train_epoch(summary, summary_dev, cfg, model, dataloader,
                dataloader_dev, optimizer, summary_writer, best_dict,
                dev_header):
    torch.set_grad_enabled(True)
    model.train()
    
    device = torch.device('cuda')
    steps = len(dataloader)
    dataiter = iter(dataloader)
    num_tasks = 2
    label_header = dataloader.dataset._label_header
    #num_tasks = len(cfg.num_classes)

    time_now = time.time()
    loss_sum = np.zeros(num_tasks)
    acc_sum = np.zeros(num_tasks)
    for step in tqdm.tqdm(range(steps)):
        image, target = next(dataiter)
        image = image.to(device)
        target = target.to(device)
        output, logit_map = model(image)

        # different number of tasks
        loss = 0
        for t in range(num_tasks):
            loss_t, acc_t = get_loss(output, target, t, device, cfg)
            loss += loss_t
            loss_sum[t] += loss_t.item()
            acc_sum[t] += acc_t.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        summary['step'] += 1

        if summary['step'] % cfg.log_every == 0:
            time_spent = time.time() - time_now
            time_now = time.time()

            loss_sum /= cfg.log_every
            acc_sum /= cfg.log_every
            loss_str = ' '.join(map(lambda x: '{:.5f}'.format(x), loss_sum))
            acc_str = ' '.join(map(lambda x: '{:.3f}'.format(x), acc_sum))

            logging.info(
                '{}, Train, Epoch : {}, Step : {}, Loss : {}, '
                'Acc : {}, Run Time : {:.2f} sec'
                .format(time.strftime("%Y-%m-%d %H:%M:%S"),
                        summary['epoch'] + 1, summary['step'], loss_str,
                        acc_str, time_spent))

            for t in range(num_tasks):
                summary_writer.add_scalar(
                    'train/loss_{}'.format(label_header[t]), loss_sum[t],
                    summary['step'])
                summary_writer.add_scalar(
                    'train/acc_{}'.format(label_header[t]), acc_sum[t],
                    summary['step'])

            loss_sum = np.zeros(num_tasks)
            acc_sum = np.zeros(num_tasks)

        if summary['step'] % cfg.test_every == 0:
            time_now = time.time()
            summary_dev, predlist, true_list = test_epoch(
                summary_dev, cfg, model, dataloader_dev)
            time_spent = time.time() - time_now

            auclist = []
            for i in range(len(cfg.num_classes)):
                y_pred = predlist[i]
                y_true = true_list[i]
                fpr, tpr, thresholds = metrics.roc_curve(
                    y_true, y_pred, pos_label=1)
                auc = metrics.auc(fpr, tpr)
                auclist.append(auc)
            summary_dev['auc'] = np.array(auclist)

            loss_dev_str = ' '.join(map(lambda x: '{:.5f}'.format(x),
                                        summary_dev['loss']))
            acc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                                       summary_dev['acc']))
            auc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                                       summary_dev['auc']))

            logging.info(
                '{}, Dev, Step : {}, Loss : {}, Acc : {}, Auc : {},'
                'Mean auc: {:.3f} ''Run Time : {:.2f} sec' .format(
                    time.strftime("%Y-%m-%d %H:%M:%S"),
                    summary['step'],
                    loss_dev_str,
                    acc_dev_str,
                    auc_dev_str,
                    summary_dev['auc'].mean(),
                    time_spent))

            for t in range(len(cfg.num_classes)):
                summary_writer.add_scalar(
                    'dev/loss_{}'.format(dev_header[t]),
                    summary_dev['loss'][t], summary['step'])
                summary_writer.add_scalar(
                    'dev/acc_{}'.format(dev_header[t]), summary_dev['acc'][t],
                    summary['step'])
                summary_writer.add_scalar(
                    'dev/auc_{}'.format(dev_header[t]), summary_dev['auc'][t],
                    summary['step'])

            save_best = False
            mean_acc = summary_dev['acc'][cfg.save_index].mean()
            if mean_acc >= best_dict['acc_dev_best']:
                best_dict['acc_dev_best'] = mean_acc
                if cfg.best_target == 'acc':
                    save_best = True

            mean_auc = summary_dev['auc'][cfg.save_index].mean()
            if mean_auc >= best_dict['auc_dev_best']:
                best_dict['auc_dev_best'] = mean_auc
                if cfg.best_target == 'auc':
                    save_best = True

            mean_loss = summary_dev['loss'][cfg.save_index].mean()
            if mean_loss <= best_dict['loss_dev_best']:
                best_dict['loss_dev_best'] = mean_loss
                if cfg.best_target == 'loss':
                    save_best = True

            if save_best:
                torch.save(
                    {'epoch': summary['epoch'],
                     'step': summary['step'],
                     'acc_dev_best': best_dict['acc_dev_best'],
                     'auc_dev_best': best_dict['auc_dev_best'],
                     'loss_dev_best': best_dict['loss_dev_best'],
                     'state_dict': model.state_dict()},
                    os.path.join(cfg.save_path, 'best{}.tar'.format(
                        best_dict['best_idx']))
                )
                best_dict['best_idx'] += 1
                if best_dict['best_idx'] > cfg.save_top_k:
                    best_dict['best_idx'] = 1
                logging.info(
                    '{}, Best, Step : {}, Loss : {}, Acc : {},Auc :{},'
                    'Best Auc : {:.3f}' .format(
                        time.strftime("%Y-%m-%d %H:%M:%S"),
                        summary['step'],
                        loss_dev_str,
                        acc_dev_str,
                        auc_dev_str,
                        best_dict['auc_dev_best']))
        model.train()
        torch.set_grad_enabled(True)
    summary['epoch'] += 1

    return summary, best_dict


In [22]:
def test_epoch(summary, cfg, model, dataloader):
    torch.set_grad_enabled(False)
    model.eval()

    device = torch.device('cuda')
    steps = len(dataloader)
    dataiter = iter(dataloader)
    num_tasks = len(cfg.num_classes)

    loss_sum = np.zeros(num_tasks)
    acc_sum = np.zeros(num_tasks)

    predlist = list(x for x in range(len(cfg.num_classes)))
    true_list = list(x for x in range(len(cfg.num_classes)))
    for step in range(steps):
        image, target = next(dataiter)
        image = image.to(device)
        target = target.to(device)
        output, logit_map = model(image)
        # different number of tasks
        for t in range(len(cfg.num_classes)):

            loss_t, acc_t = get_loss(output, target, t, device, cfg)
            # AUC
            output_tensor = torch.sigmoid(
                output[t].view(-1)).cpu().detach().numpy()
            target_tensor = target[:, t].view(-1).cpu().detach().numpy()
            if step == 0:
                predlist[t] = output_tensor
                true_list[t] = target_tensor
            else:
                predlist[t] = np.append(predlist[t], output_tensor)
                true_list[t] = np.append(true_list[t], target_tensor)

            loss_sum[t] += loss_t.item()
            acc_sum[t] += acc_t.item()
    summary['loss'] = loss_sum / steps
    summary['acc'] = acc_sum / steps
    print(predlist)
    print(len(predlist))
    print(predlist[0].shape)
    return summary, predlist, true_list

In [23]:
def get_norm(norm_type, num_features, num_groups=32, eps=1e-5):
    if norm_type == 'BatchNorm':
        return nn.BatchNorm2d(num_features, eps=eps)
    elif norm_type == "GroupNorm":
        return nn.GroupNorm(num_groups, num_features, eps=eps)
    elif norm_type == "InstanceNorm":
        return nn.InstanceNorm2d(num_features, eps=eps,
                                 affine=True, track_running_stats=True)
    else:
        raise Exception('Unknown Norm Function : {}'.format(norm_type))


def get_optimizer(params, cfg):
    if cfg.optimizer == 'SGD':
        return SGD(params, lr=cfg.lr, momentum=cfg.momentum,
                   weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'Adadelta':
        return Adadelta(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'Adagrad':
        return Adagrad(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'Adam':
        return Adam(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'RMSprop':
        return RMSprop(params, lr=cfg.lr, momentum=cfg.momentum,
                       weight_decay=cfg.weight_decay)
    else:
        raise Exception('Unknown optimizer : {}'.format(cfg.optimizer))


In [ ]:

from model.classifier import Classifier

if not os.path.exists(cfg.save_path):
    os.mkdir(cfg.save_path)
if cfg.logtofile is True:
    logging.basicConfig(filename=cfg.save_path + '/log.txt',
                        filemode="w", level=logging.INFO)
else:
    logging.basicConfig(level=logging.INFO)





device = torch.device('cuda')

model = Classifier(cfg)
if cfg.verbose is True:
    from torchsummary import summary
    if cfg.fix_ratio:
        h, w = cfg.long_side, cfg.long_side
    else:
        h, w = cfg.height, cfg.width
    summary(model.to(device), (3, h, w))
model = model.to(device).train()
if cfg.pre_train is not None:
    if os.path.exists(cfg.pre_train):
        ckpt = torch.load(cfg.pre_train, map_location=device)
        model.module.load_state_dict(ckpt)
optimizer = get_optimizer(model.parameters(), cfg)



dataloader_train = DataLoader(
    CheXDataset(cfg.train_csv, diseases, data_aug_transforms),
    batch_size=cfg.train_batch_size, num_workers=0,
    drop_last=True, shuffle=True)
dataloader_dev = DataLoader(
    CheXDataset(cfg.dev_csv, diseases),
    batch_size=cfg.dev_batch_size, num_workers=0,
    drop_last=False, shuffle=False)
dev_header = dataloader_dev.dataset._label_header

summary_train = {'epoch': 0, 'step': 0}
summary_dev = {'loss': float('inf'), 'acc': 0.0}
summary_writer = SummaryWriter(cfg.save_path)
epoch_start = 0
best_dict = {
    "acc_dev_best": 0.0,
    "auc_dev_best": 0.0,
    "loss_dev_best": float('inf'),
    "fused_dev_best": 0.0,
    "best_idx": 1}

if cfg.resume:
    ckpt_path = os.path.join(cfg.save_path, 'train.tar')
    ckpt = torch.load(ckpt_path, map_location=device)
    model.module.load_state_dict(ckpt['state_dict'])
    summary_train = {'epoch': ckpt['epoch'], 'step': ckpt['step']}
    best_dict['acc_dev_best'] = ckpt['acc_dev_best']
    best_dict['loss_dev_best'] = ckpt['loss_dev_best']
    best_dict['auc_dev_best'] = ckpt['auc_dev_best']
    epoch_start = ckpt['epoch']

for epoch in range(epoch_start, cfg.epoch):
    lr = lr_schedule(cfg.lr, cfg.lr_factor, summary_train['epoch'],
                     cfg.lr_epochs)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    summary_train, best_dict = train_epoch(
        summary_train, summary_dev, cfg, model,
        dataloader_train, dataloader_dev, optimizer,
        summary_writer, best_dict, dev_header)

    time_now = time.time()
    summary_dev, predlist, true_list = test_epoch(
        summary_dev, cfg, model, dataloader_dev)
    time_spent = time.time() - time_now

    auclist = []
    for i in range(len(cfg.num_classes)):
        y_pred = predlist[i]
        y_true = true_list[i]
        fpr, tpr, thresholds = metrics.roc_curve(
            y_true, y_pred, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        auclist.append(auc)
    summary_dev['auc'] = np.array(auclist)

    loss_dev_str = ' '.join(map(lambda x: '{:.5f}'.format(x),
                                summary_dev['loss']))
    acc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                               summary_dev['acc']))
    auc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                               summary_dev['auc']))

    logging.info(
        '{}, Dev, Step : {}, Loss : {}, Acc : {}, Auc : {},'
        'Mean auc: {:.3f} ''Run Time : {:.2f} sec' .format(
            time.strftime("%Y-%m-%d %H:%M:%S"),
            summary_train['step'],
            loss_dev_str,
            acc_dev_str,
            auc_dev_str,
            summary_dev['auc'].mean(),
            time_spent))

    for t in range(len(cfg.num_classes)):
        summary_writer.add_scalar(
            'dev/loss_{}'.format(dev_header[t]), summary_dev['loss'][t],
            summary_train['step'])
        summary_writer.add_scalar(
            'dev/acc_{}'.format(dev_header[t]), summary_dev['acc'][t],
            summary_train['step'])
        summary_writer.add_scalar(
            'dev/auc_{}'.format(dev_header[t]), summary_dev['auc'][t],
            summary_train['step'])

    save_best = False

    mean_acc = summary_dev['acc'][cfg.save_index].mean()
    if mean_acc >= best_dict['acc_dev_best']:
        best_dict['acc_dev_best'] = mean_acc
        if cfg.best_target == 'acc':
            save_best = True

    mean_auc = summary_dev['auc'][cfg.save_index].mean()
    if mean_auc >= best_dict['auc_dev_best']:
        best_dict['auc_dev_best'] = mean_auc
        if cfg.best_target == 'auc':
            save_best = True

    mean_loss = summary_dev['loss'][cfg.save_index].mean()
    if mean_loss <= best_dict['loss_dev_best']:
        best_dict['loss_dev_best'] = mean_loss
        if cfg.best_target == 'loss':
            save_best = True

    if save_best:
        torch.save(
            {'epoch': summary_train['epoch'],
             'step': summary_train['step'],
             'acc_dev_best': best_dict['acc_dev_best'],
             'auc_dev_best': best_dict['auc_dev_best'],
             'loss_dev_best': best_dict['loss_dev_best'],
             'state_dict': model.state_dict()},
            os.path.join(cfg.save_path,
                         'best{}.tar'.format(best_dict['best_idx']))
        )
        best_dict['best_idx'] += 1
        if best_dict['best_idx'] > cfg.save_top_k:
            best_dict['best_idx'] = 1
        logging.info(
            '{}, Best, Step : {}, Loss : {}, Acc : {},'
            'Auc :{},Best Auc : {:.3f}' .format(
                time.strftime("%Y-%m-%d %H:%M:%S"),
                summary_train['step'],
                loss_dev_str,
                acc_dev_str,
                auc_dev_str,
                best_dict['auc_dev_best']))
    torch.save({'epoch': summary_train['epoch'],
                'step': summary_train['step'],
                'acc_dev_best': best_dict['acc_dev_best'],
                'auc_dev_best': best_dict['auc_dev_best'],
                'loss_dev_best': best_dict['loss_dev_best'],
                'state_dict': model.state_dict()},
               os.path.join(cfg.save_path, 'train.tar'))
summary_writer.close()

  1%|▌                                                                           | 100/13963 [01:19<9:28:43,  2.46s/it]

[array([0.40865415, 0.33929217, 0.5532701 , 0.3409559 , 0.33275056,
       0.51809955, 0.45247072, 0.33935985, 0.3713418 , 0.55725145,
       0.59062606, 0.4335949 , 0.31627908, 0.3429036 , 0.4911856 ,
       0.5905155 , 0.38563937, 0.3746953 , 0.3914629 , 0.3387645 ,
       0.58145326, 0.41599518, 0.29697654, 0.44481182, 0.36883268,
       0.56519026, 0.33270374, 0.5218934 , 0.32134292, 0.29105127,
       0.34263653, 0.43058664, 0.4630524 , 0.41843393, 0.6204558 ,
       0.40878052, 0.49686444, 0.31224078, 0.42851847, 0.38025975,
       0.51908565, 0.4570511 , 0.41802582, 0.4138871 , 0.44770256,
       0.3914962 , 0.45124313, 0.37998903, 0.33997178, 0.36118025,
       0.37826633, 0.46188298, 0.29294702, 0.5703491 , 0.32359523,
       0.5455836 , 0.32189146, 0.4003335 , 0.565692  , 0.46703717,
       0.386882  , 0.52064866, 0.39924756, 0.650126  , 0.5305676 ,
       0.42976746, 0.32653484, 0.6442597 , 0.34879246, 0.42799544,
       0.37454513, 0.37092268, 0.5002455 , 0.5435003 , 0.6764

  1%|█                                                                           | 200/13963 [02:39<9:26:53,  2.47s/it]

[array([0.40339604, 0.25150424, 0.41524252, 0.27671322, 0.3018205 ,
       0.4632096 , 0.4574401 , 0.2785367 , 0.2509669 , 0.49551857,
       0.52088356, 0.46726498, 0.31492686, 0.321896  , 0.4137753 ,
       0.5676903 , 0.32232657, 0.3829117 , 0.31615084, 0.34699473,
       0.48101857, 0.44241184, 0.2207885 , 0.4187824 , 0.3349496 ,
       0.42908964, 0.3055294 , 0.5191462 , 0.3167354 , 0.25199226,
       0.34575912, 0.37717238, 0.48490667, 0.38210794, 0.53687555,
       0.29890814, 0.49718884, 0.179881  , 0.36631122, 0.3281406 ,
       0.511005  , 0.43673724, 0.40540132, 0.43522525, 0.46133786,
       0.296609  , 0.40245193, 0.31412378, 0.3275384 , 0.3569684 ,
       0.33372176, 0.4349246 , 0.27464226, 0.49359643, 0.30034718,
       0.5169148 , 0.30194998, 0.3126776 , 0.53636134, 0.42334858,
       0.39556426, 0.39031294, 0.40130866, 0.5584862 , 0.48825088,
       0.4969004 , 0.29844892, 0.49874336, 0.3071411 , 0.4843941 ,
       0.36397552, 0.26509756, 0.44043973, 0.45825285, 0.6120

  2%|█▋                                                                          | 300/13963 [03:54<9:10:35,  2.42s/it]

...

 14%|██████████▋                                                                | 2000/13963 [24:59<8:12:15,  2.47s/it]

[array([0.37029108, 0.12292152, 0.1117933 , 0.43425375, 0.14990021,
       0.40608263, 0.40596136, 0.22495754, 0.2801439 , 0.26738402,
       0.44462442, 0.67537767, 0.24530645, 0.29691735, 0.18236125,
       0.5978119 , 0.4645348 , 0.32889825, 0.42195988, 0.34474736,
       0.23149367, 0.75861   , 0.248451  , 0.3585407 , 0.22437356,
       0.22492248, 0.4693272 , 0.33922762, 0.24186127, 0.09208934,
       0.1747299 , 0.33132246, 0.62994665, 0.29170653, 0.2545519 ,
       0.407799  , 0.22676562, 0.14122371, 0.6193883 , 0.5782734 ,
       0.5899013 , 0.36937264, 0.41016218, 0.38884574, 0.33850294,
       0.30879885, 0.1497148 , 0.61499155, 0.42639315, 0.5754236 ,
       0.5250461 , 0.47538546, 0.2761799 , 0.30239245, 0.29798502,
       0.6254591 , 0.15028472, 0.29155284, 0.5084539 , 0.46578753,
       0.37631798, 0.54705864, 0.46081853, 0.42477906, 0.41323614,
       0.47798106, 0.74751085, 0.8705907 , 0.21590416, 0.73103774,
       0.39797682, 0.1861603 , 0.1956286 , 0.32692164, 0.3726

 15%|███████████▏                                                              | 2100/13963 [27:24<28:02:59,  8.51s/it]

[array([0.281087  , 0.07509189, 0.06996349, 0.5071198 , 0.08629093,
       0.23773114, 0.39113688, 0.3826263 , 0.43827364, 0.28864643,
       0.6061087 , 0.7351288 , 0.12557873, 0.1752453 , 0.19161378,
       0.6276588 , 0.36439407, 0.20022742, 0.56365305, 0.20565887,
       0.23828803, 0.6778657 , 0.17782953, 0.21706493, 0.09844698,
       0.18665993, 0.3920067 , 0.48863694, 0.16417733, 0.08165167,
       0.13535477, 0.35335878, 0.6372807 , 0.16803582, 0.25675145,
       0.43682662, 0.23487572, 0.20569018, 0.49052206, 0.5279136 ,
       0.4049257 , 0.40485215, 0.31755248, 0.29191437, 0.2592949 ,
       0.5607879 , 0.12797798, 0.21585818, 0.32416597, 0.23142909,
       0.68454033, 0.33424297, 0.44098505, 0.26742515, 0.4109684 ,
       0.37091383, 0.10564634, 0.18410522, 0.18695894, 0.27179652,
       0.23948675, 0.39322963, 0.22113612, 0.4986044 , 0.48688093,
       0.36906296, 0.7042867 , 0.525224  , 0.12960464, 0.8074672 ,
       0.20010905, 0.2185213 , 0.16582882, 0.21057984, 0.5135

 15%|███████████▍                                                               | 2127/13963 [28:08<5:46:46,  1.76s/it]

In [ ]:
cfg.save_path

In [ ]:
cfg.logtofile